<a href="https://colab.research.google.com/github/Abhishek0697/Detection-of-Hate-Speech-in-Multimodal-Memes/blob/main/Code/Experiments/BERT%2BResNet_Late_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
'''
IMPORTING NECESSARY MODULES
'''
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('./Trainers/')
sys.path.append('./Dataloaders/')
sys.path.append('./utils/')
sys.path.append('./architectures/')

from dataloader import mydataset, create_prime_dict 
from Load_model import load
from plot_curves import plot_loss, plot_acc
from trainer_Resnet_BERT_late_fusion import train, test_classify

'''
For ResNet
'''
from resnet_models import ResNet,Bottleneck

'''
For BERT
'''
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

**Device**

In [ ]:
gpu_ids = [7]
device = torch.device('cuda:'+ str(gpu_ids[0]) if torch.cuda.is_available() else 'cpu')
print(device)

**Dataloading Scheme**

In [ ]:
trainlist = 'add path of training list'
validlist = 'add path of validation list'

In [ ]:
classes = create_prime_dict(trainlist)

In [ ]:
'''
Train Dataloader
''' 
train_dataset = mydataset(trainlist, classes, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 64, num_workers=16,pin_memory=True)


'''
Validation Dataloader
''' 
validation_dataset = mydataset(validlist, classes, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 64, num_workers=16,pin_memory=True)

**Model Definition**

In [ ]:
'''
Model1 ResNet50
'''
Image_model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 2)
Image_model = nn.DataParallel(Image_model,device_ids = gpu_ids).to(device)


'''
Load saved model from checkpoint
'''
model1_name = 'Resnet50'
model1_path = './saved_model_checkpoints/'+model1_name

checkpoint1 = torch.load(model1_path)
Image_model.load_state_dict(checkpoint1['model_state_dict'])

Image_model.to(device)

In [ ]:
'''
Model 2 BERT

Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
''' 

Text_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2,   
    output_attentions = False, 
    output_hidden_states = True
)

Text_model = nn.DataParallel(Text_model,device_ids=gpu_ids).to(device)


'''
Load saved model from checkpoint
'''
model2_name = 'BERT_basic'
model2_path = './saved_model_checkpoints/'+model2_name

checkpoint2 = torch.load(model2_path)
Text_model.load_state_dict(checkpoint2['model_state_dict'])

Text_model.to(device)

In [ ]:
'''
Fusion
'''
class FusionNet(nn.Module):
    
    def __init__(self, num_classes, drop_prob = 0.1):
        super(FusionNet, self).__init__()
        
        self.pooler = nn.Linear(in_features=768, out_features=768)
        
        self.concat = nn.Linear(in_features=768+2048, out_features= 512)
                
        self.classify = nn.Linear(in_features = 512, out_features = num_classes)
        
        
    def forward(self, text_features, image_features):

        x =(text_features+image_features)/2

        return x

In [ ]:
Fusion_model = FusionNet(num_classes =2 , drop_prob = 0.1)
Fusion_model = nn.DataParallel(Fusion_model, device_ids=gpu_ids).to(device)

**Hyperparameters**

In [ ]:
'''
Loss Function
'''
criterion = nn.CrossEntropyLoss()


'''
Optimizer
'''
optimizer = torch.optim.SGD(Fusion_model.parameters(), lr=0.01, weight_decay=1e-4, momentum=0.9)
# optimizer = AdamW(Fusion_model.parameters(), lr = 2e-3, eps = 1e-8)


'''
Number of training epochs.
'''
num_Epochs = 10


# '''
# OneCycleLR
# '''
# max_lr = 0.05
# lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, total_steps=None, epochs=num_Epochs, steps_per_epoch=len(train_dataloader), pct_start=0.3, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=25.0, final_div_factor=10000.0, last_epoch=-1)


lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 4, gamma = 0.1)

# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)

In [ ]:
model_name = 'Late_Fusion'
model_path = './saved_model_checkpoints/'+model_name

In [ ]:
writer = SummaryWriter(model_name)

train(Image_model, Text_model, Fusion_model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, model_path, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

In [ ]:
'''
Load saved model from checkpoint
'''
Fusion_model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(model_path, Fusion_model, optimizer, lr_scheduler)


**Evaluate**

In [ ]:
test_classify(Fusion_model, validation_dataloader, criterion, device)